## LIBRERIAS 
## DEPENDENCIES

In [2]:
import pandas as pd
import folium

from geopy.geocoders import Nominatim
from geopy import distance
import math
import json
import geopandas as gdp
from folium.plugins import Search
from folium.plugins import MarkerCluster

In [ ]:
# https://towardsdatascience.com/your-cool-folium-maps-on-the-web-313f9d1a6bcd

## Importar data

In [3]:
colegios = pd.read_csv('./data/data_coles_complete_gm.csv', index_col=0)
resis = pd.read_csv('./data/data_resis_complete_gm.csv')

## Limpieza de datos (preprocesado)
## Preprocessing data


In [69]:
# quitar S/N (sin numero en ciertas direcciones)
calles = []

for i in range(0,len(colegios)):
    if 'S/n' in colegios.iloc[i]['DOMICILIO']:
        calles.append(colegios.iloc[i]['DOMICILIO'].replace(', S/n', ' '))
    else: 
        calles.append(colegios.iloc[i]['DOMICILIO'])

print(len(calles))
colegios['calle'] = calles

4888


## GEOCODING

In [29]:
geolocator = Nominatim(user_agent="example app")

In [10]:
# buscar latitudes y longitudes 

def get_coordinates(df, street_column, postal_code):
    latitudes = []
    longitudes = []
    for i in range(0,len(df)):
        try: 
            x = geolocator.geocode(df.iloc[i][street_column]+', Madrid, '+df.iloc[i][postal_code]).point
            latitudes.append(x[0])
            longitudes.append(x[1])
        except: 
            latitudes.append('N/A')
            longitudes.append('N/A')

    return latitudes, longitudes


In [ ]:
latitudes, longitudes = get_coordinates(colegios, 'calle', 'CODIGO POST.')

In [32]:
colegios['latitudes'] = latitudes
colegios['longitudes'] = longitudes

Hacer lo mismo para las residencias 

In [125]:
latitudes, longitudes = get_coordinates(resis, 'Calle', 'CP')
resis['latitudes'] = latitudes
resis['longitudes'] = longitudes

In [ ]:
# resis.to_csv('./data/data_resis_complete.csv')
# colegios.to_csv('./data/data_coles_complete.csv')

## Dealing with outliers (no los pillaba bien el geocoder- los ponia fuera de madrid)

In [13]:
colegios[colegios['CENTRO'] == 'E.O.I. DE RIVAS-VACIAMADRID']

,Unnamed: 0.1,AREA TERRITORIAL,CODIGO CENTRO,TIPO DE CENTRO,CENTRO,DOMICILIO,MUNICIPIO,DISTRITO MUNICIPAL,COD. POSTAL,TELEFONO,FAX,EMAIL,EMAIL2,TITULARIDAD,TIPO,latitudes,longitudes,calle
1861,1862,Madrid-Este,28059310,EOI,E.O.I. DE RIVAS-VACIAMADRID,Calle De Los Picos De Urbión C/v A C/ Jaen,Rivas-Vaciamadrid,-,28522,914990537,914990623,eoi.rivas@educa.madrid.org,-,Público,Primaria,37.779594,-3.784906,Calle De Los Picos De Urbión C/v A C/ Jaen


In [375]:
colegios[colegios['CENTRO'] == 'LA LAGUNA']['latitudes']

1646    40.801125
4221    40.801125
Name: latitudes, dtype: float64

In [14]:
colegios.iloc[1861]['latitudes']

37.7795941

In [16]:
colegios.iloc[1861]['longitudes']

-3.784905699999999

In [ ]:
40.37408793380588, -3.5272361053978

In [17]:
# changing lat/long of TOPITOS

colegios.replace(37.7795941, 40.37408793380588, inplace=True)
colegios.replace(-3.784905699999999, -3.5272361053978, inplace=True)

In [163]:
# changing lat/long of Las cumbres

colegios.replace(38.5468246, 40.55409582358531, inplace=True)
colegios.replace(-0.12771876388221354, -3.630698196842926, inplace=True)

In [168]:
# changing lat/long of GRIMM

colegios.replace(36.4635133, 40.36227667986738, inplace=True)
colegios.replace(-5.0413245, -3.5329390312277527, inplace=True)

## Dibujar el Mapa
## Drawing on Map


In [30]:
colegios

,Unnamed: 0.1,AREA TERRITORIAL,CODIGO CENTRO,TIPO DE CENTRO,CENTRO,DOMICILIO,MUNICIPIO,DISTRITO MUNICIPAL,COD. POSTAL,TELEFONO,FAX,EMAIL,EMAIL2,TITULARIDAD,TIPO,latitudes,longitudes,calle
0,1,Madrid-Oeste,28060646,EEI,ACHALAY,"Avenida De Isabel De Farnesio, 14",Boadilla del Monte,-,28660,916326518,-,eei.achalay.boadilla@educa.madrid.org,achalay-@hotmail.com,Público,Primaria,40.403939,-3.887527,"Avenida De Isabel De Farnesio, 14"
1,2,Madrid-Este,28063027,EEI,ACUARELA,"Avenida Del Somorrostro, 193",San Fernando de Henares,-,28830,916694580,-,eei.acuarela.sanfernando@educa.madrid.org,-,Público,Primaria,40.414520,-3.521631,"Avenida Del Somorrostro, 193"
2,3,Madrid-Sur,28047460,EEI,ADIVINANZAS,Plaza Duque De Ahumada,Alcorcón,-,28922,911886061,-,eei.adivinanzas.alcorcon@educa.madrid.org,-,Público,Primaria,40.189136,-3.680518,Plaza Duque De Ahumada
3,4,Madrid-Capital,28047551,EEI,AGUA DULCE,"Calle De Leñeros, 25",Madrid,Tetuán,28039,914594049,914594049,eei.aguadulce.madrid@educa.madrid.org,-,Público,Primaria,40.453890,-3.709600,"Calle De Leñeros, 25"
4,5,Madrid-Sur,28044100,EEI,AILANTOS,"Plaza Duque De Ahumada, S/n",Valdemoro,-,28341,918955900,918955900,eei.ailantos.valdemoro@educa.madrid.org,manuelaailantos@hotmail.com,Público,Primaria,40.189136,-3.680518,Plaza Duque De Ahumada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,839,Madrid-Oeste,28030681,CPR PRI-SEC,SAN MIGUEL II,"Calle Río Guadiana, 2",Boadilla del Monte,-,28669,916330078,916322527,dtecnica@stmichelsschool.es,stmichaels@stmichaelsschool.es,Privado,Secundaria,40.430545,-3.919046,"Calle Río Guadiana, 2"
4884,840,Madrid-Capital,28009677,CPR PRI-SEC,SANTA MARIA DEL VALLE CEP,"Calle De Vitruvio, 14",Madrid,Chamartín,28006,915612093,-,direccionvitruvio@santamariadelvalle.es,secretariavitruvio@santamariadelvalle.es,Privado,Secundaria,40.443679,-3.687501,"Calle De Vitruvio, 14"
4885,841,Madrid-Sur,28029445,CPR PRI-SEC,SEMINARIO ARZOBISPAL LA INMACULADA Y SAN DAMASO,"Carretera Alcorcon-plasencia, K. 67",Rozas de Puerto Real,-,28649,918610200,918611356,colegioseminario@rozascs.org,secretaria@rozascs.org,Privado,Secundaria,40.302290,-4.556260,"Carretera Alcorcon-plasencia, K. 67"
4886,842,Madrid-Capital,28076927,CPR PRI-SEC,STELLA MARIS COLLEGE & PREP. SCHOOL,"Calle De Valdesquí, 16",Madrid,Moncloa-Aravaca,28023,910882353,-,secretaria@stellamariscollege.es,-,Privado,Secundaria,40.467992,-3.774533,"Calle De Valdesquí, 16"


#### map version 1

In [15]:
m = folium.Map(location=[40.380708,-3.741548], prefer_canvas=True)

# optional argument zoom_start=5

In [16]:
for i in range(0,len(resis)):
    try:
        folium.Marker(location=[resis.iloc[i]['latitudes'],resis.iloc[i]['longitudes']], popup=resis.iloc[i]['Residencia'], icon=folium.Icon(color='black', icon_color='#bdd2c1') ).add_to(m)
    except: 
         i = i 

In [17]:
for i in range(0,len(colegios)):
    try:
        folium.CircleMarker(location=[colegios.iloc[i]['latitudes'],colegios.iloc[i]['longitudes']], popup=colegios.iloc[i]['CENTRO'], radius=7, color="#f0985d", fill_color="#f0985d"  ).add_to(m)
    except: 
        i = i 

### colors : #bdd2c1, #596a62, #f0985d

### marker cluster map 

In [41]:
resi_mas_cercana = pd.read_csv('./data/resi_mas_cercana.csv')
n_coles_cercanos = pd.read_csv('./data/n_coles_cercanos.csv')
numero_coles = n_coles_cercanos['n. coles cercanos']
dist_array = resi_mas_cercana['distance']
resi_array = resi_mas_cercana['residencia']

In [13]:
f = folium.Map(location=[40.380708,-3.741548], prefer_canvas=True)
# zoom_start 

marker_cluster = MarkerCluster(name="Cluster").add_to(f)

In [54]:
for i in range(0,len(colegios)):
    try:
        dist = round(dist_array[i],2)
        if dist <= 5:
            folium.CircleMarker(location=[colegios.iloc[i]['latitudes'],colegios.iloc[i]['longitudes']], 
            popup= ( colegios.iloc[i]['CENTRO']+' Residencia mas cercana a: '+str(dist)+', '+ resi_array[i]), 
            radius=10, color='green', 
            fill_color="green", 
            label =(colegios.iloc[i]['CENTRO']).lower()  ).add_to(marker_cluster)
        else:
            folium.CircleMarker(location=[colegios.iloc[i]['latitudes'],colegios.iloc[i]['longitudes']], 
            popup= ( colegios.iloc[i]['CENTRO']+' Residencia mas cercana a: '+str(dist)+', '+resi_array[i]), 
            radius=10, color='black', 
            fill_color="red", 
            label =(colegios.iloc[i]['CENTRO']).lower()  ).add_to(marker_cluster)
    except: 
        x = i

In [18]:
for i in range(0,len(resis)):
    try:
        name = resis.iloc[i]['Residencia'].upper()+', '+str(numero_coles[i])+' colegios a 5km o menos.'
        folium.Marker(
            location=[resis.iloc[i]['latitudes'],resis.iloc[i]['longitudes']], 
            popup=name, 
            icon=folium.Icon(color='black', icon_color='#bdd2c1'), 
            label =(resis.iloc[i]['Residencia']).lower()
            ).add_to(f)
    except: 
        try:
            folium.Marker(
                location=[resis.iloc[i]['latitudes'],resis.iloc[i]['longitudes']], 
                popup='No hay colegios a menos de 5km', 
                icon=folium.Icon(color='black', 
                icon_color='#bdd2c1'), 
                label =(resis.iloc[i]['Residencia']).lower()
                ).add_to(f)
        except: 
            i = i 

In [38]:
folium.Marker(
                    location=[colegios.iloc[i]['latitudes'],colegios.iloc[1]['longitudes']], 
                    popup= ( colegios.iloc[i]['CENTRO']+' Residencia mas cercana a: '+str(dist)+', '+resis.iloc[resi_number[1]]['Residencia']), 
                    label =(colegios.iloc[i]['CENTRO']).lower() , 
                    icon = icon).add_to(marker_cluster)

In [63]:
str(round(dist_array[1],2))

'3.7'

In [52]:
def plot_map():
    # create map
    plot_locations_map = folium.Map(location=[40.380708,-3.741548], zoom_start=7, tiles=None, overlay = False)

    marker_cluster = MarkerCluster(name="Colegios").add_to(plot_locations_map)
    feature_group = folium.FeatureGroup(name='Residencias',overlay=True)

    feature_group.add_to(plot_locations_map)

    for i in range(0,len(colegios)):
        try:
            dist = round(dist_array[i],2)
            if dist <= 5:
                icon_path = "./images/book_icon.png"
                icon = folium.features.CustomIcon(icon_image=icon_path ,icon_size=(31,31))
                folium.Marker(
                    location=[colegios.iloc[i]['latitudes'],colegios.iloc[i]['longitudes']], 
                    popup= ( colegios.iloc[i]['CENTRO']+' Residencia mas cercana a: '+str(dist)+', '+ resi_array[i]), 
                    label =(colegios.iloc[i]['CENTRO']).lower() , 
                    icon = icon).add_to(marker_cluster)
            else:
                icon_path = "./images/red_icon.png"
                icon2 = folium.features.CustomIcon(icon_image=icon_path ,icon_size=(31,27))
                folium.Marker(
                    location=[colegios.iloc[i]['latitudes'],colegios.iloc[i]['longitudes']], 
                    popup= ( colegios.iloc[i]['CENTRO']+' Residencia mas cercana a: '+str(dist)+', '+ resi_array[i]), 
                    label =(colegios.iloc[i]['CENTRO']).lower(),
                    icon = icon2).add_to(marker_cluster)
        except: 
            x = i


    for i in range(0,len(resis)):
        try:
            name = resis.iloc[i]['Residencia'].upper()+' '+ str(numero_coles[i]) +', colegios a 5km o menos.'
            folium.Marker(
                location=[resis.iloc[i]['latitudes'],resis.iloc[i]['longitudes']], 
                popup=name, 
                icon=folium.Icon(color='beige', icon_color='white', icon = 'home'), 
                label =(resis.iloc[i]['Residencia']).lower()
                ).add_child(feature_group)
        except: 
            try:
                folium.Marker(
                    location=[resis.iloc[i]['latitudes'],resis.iloc[i]['longitudes']], 
                    popup='No hay colegios a menos de 5km', 
                    icon=folium.Icon(color='beige', 
                    icon_color='#bdd2c1', icon = 'home'), 
                    label =(resis.iloc[i]['Residencia']).lower()
                    ).add_child(feature_group)
            except: 
                i = i 
                
    folium.TileLayer('openstreetmap',overlay=False,name = f'Toggle').add_to(plot_locations_map)
    folium.LayerControl().add_to(plot_locations_map)
    
    return plot_locations_map

In [68]:
# using ipywidgets
# plot location with marker

import geocoder
import ipywidgets

# text widget
address_text_box = ipywidgets.Text(value='', placeholder='escribe aqui', description='dirección:')

# widget function
def plot_locations(address):
    # location address
    location = geocoder.osm(address)
    
    # latitude and longitude of location
    latlng = [location.lat, location.lng]
    
    # create map
    plot_locations_map = plot_map()
    
    # marker
    folium.Marker(latlng, popup=str(address), tooltip='click').add_to(plot_locations_map)
    
    # display map
    display(plot_locations_map)
    
# interaction between widget and function  
ipywidgets.interact_manual(plot_locations, address=address_text_box)


interactive(children=(Text(value='', description='dirección:', placeholder='escribe aqui'), Button(description…

<function __main__.plot_locations(address)>

In [324]:
plot_locations_map .save('mapV4.html')

In [243]:
servicesearch = Search(
    layer=marker_cluster,
    search_label="label",
    placeholder='Search for a service',
    collapsed=False,
).add_to(f)



In [71]:
resis

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Residencia,DISTRITO,CP,Comunidad,Calle,Telefono,Telefono Alternativo,Tipo,Oficina,Estado,latitudes,longitudes
0,0,0,0,Acogida HORTALEZA,Hortaleza,28043,Madrid,"C. de Valdetorres de Jarama, 1",917642900,917642943,Primera Acogida,NaN,NaN,40.472838,-3.647690
1,1,1,1,Acogida Isabel Clara Eugenia,Hortaleza,28033,Madrid,"C. del Mar Caspio, 8",917633222,917633240,Primera Acogida,NaN,NaN,40.473036,-3.644542
2,2,2,2,Acogida casa de campo. Fundación Antonio Moreno,LATINA,28011,Madrid,"Camino del Robledal, 2",914700553,NaN,Primera Acogida,NaN,NaN,40.415284,-3.749185
3,3,3,3,Casa de los niños,"Fuencarral-El Pardo, Madrid",28049,Madrid,"Carr. Vía de Servicio, Km.12,800",917343082,917348780,Primera Infancia,NaN,NaN,40.542991,-3.703180
4,4,4,4,Residencia Menores Rosa,Chamartin,28036,Madrid,"P.º de la Habana, 173",913455173,913507392,Primera Infancia,NaN,NaN,40.465941,-3.679478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,71,71,71,Dianova España,NaN,28045,Madrid,NaN,914688764,914688765,"Centro menores con trastornos de salud mental,...","Calle Coslada, 15 – 1ºD 28028 (Madrid)",NaN,NaN,NaN
72,72,72,72,Reaps. Picón de Jarama,Paracuellos de Jarama,28860,Madrid,"C/ Santa Ana, 27",916580028,NaN,"Centro menores con trastornos de salud mental,...",NaN,NaN,40.502921,-3.532029
73,73,73,73,Santa Marta,NaN,28210,Madrid,NaN,918965968,NaN,"Centro menores con trastornos de salud mental,...",NaN,NaN,NaN,NaN
74,74,74,74,Sirio I,"C. de Amparo Usera, 44",28026,Madrid,NaN,914763372,NaN,"Centro menores con trastornos de salud mental,...",NaN,NaN,NaN,NaN


In [284]:
folium.LayerControl().add_to(f)

In [193]:
# '('+resis.iloc[i]['Tipo']+')'

'(Primera Acogida)'

# DISTANCE?

In [43]:
## Calculate the distance between schools and resis
# find the closest resi to each school
## Calcula las distancias entre las resis y los colegios
# Encuentra la resi mas cercana de cada colegio

close_dist = list(range(0,len(colegios)))
resi_number = [0] * len(colegios)

for i in range(0, len(colegios)):
    win = 1000000
    for j in range(0, len(resis)):

        lat_resi = resis.iloc[j]['latitudes']
        lon_resi = resis.iloc[j]['longitudes']
        lat_cole = colegios.iloc[i]['latitudes']
        lon_cole = colegios.iloc[i]['longitudes']

        start, end = (lat_resi,lon_resi),(lat_cole,lon_cole)

        try:
            aux = distance.distance(start, end).km

            if aux < win :
                win = aux 
                
                close_dist[i] = aux
                resi_number[i] = j

        except:
            win = win

In [256]:
# close_dist.to_csv('dist_resi_mas_cercana.csv')
# resi_number.to_csv('num_resi_mas_cercana.csv')

In [44]:
## Calculate the distance between schools and resis
# find all resis <= 5 km to a school
close_schools_dist = []
close_schools = []

for p in range(0, len(resis)):

    lat_resi = resis.iloc[p]['latitudes']
    lon_resi = resis.iloc[p]['longitudes']

    close_schools.append([])
    close_schools_dist.append([])

    if math.isnan(lat_resi) == False and math.isnan(lon_resi) == False:

        for q in range(0, len(colegios)):
            
            lat_cole = colegios.iloc[q]['latitudes']
            lon_cole = colegios.iloc[q]['longitudes']

            if lat_cole != 'N/A' and lon_cole != 'N/A':

                start, end = (lat_resi,lon_resi),(lat_cole,lon_cole)
                try: 
                    aux = distance.distance(start, end).km
                except: 
                    print(p,q,start,end)

                if aux <= 5:
                    close_schools[p].append(q)
                    close_schools_dist[p].append(aux)
                else: 
                    continue

            else: continue
    else: 
        continue


0 2119 (40.4728382, -3.6476904) (nan, nan)
1 2119 (40.4730365, -3.6445424) (nan, nan)
2 2119 (40.415284, -3.7491851) (nan, nan)
3 2119 (40.5429909, -3.7031795) (nan, nan)
4 2119 (40.4659413, -3.6794779) (nan, nan)
5 2119 (40.4407132, -3.675921299999999) (nan, nan)
6 2119 (40.3829505, -3.7348214) (nan, nan)
7 2119 (40.490647, -3.3484847) (nan, nan)
8 2119 (40.3567299, -3.8056339) (nan, nan)
9 2119 (40.0262037, -3.6140938) (nan, nan)
10 2119 (40.453577, -3.7047695) (nan, nan)
11 2119 (40.2903102, -3.4424596) (nan, nan)
12 2119 (40.6674526, -3.7655124) (nan, nan)
13 2119 (40.4349869, -3.7092035) (nan, nan)
14 2119 (40.452459, -3.6520661) (nan, nan)
15 2119 (40.4350543, -3.7047142) (nan, nan)
16 2119 (40.377128, -3.6243331) (nan, nan)
17 2119 (40.4211286, -3.6169124) (nan, nan)
18 2119 (40.3182536, -3.7943157) (nan, nan)
19 2119 (40.3178506, -3.8842256) (nan, nan)
20 2119 (40.4426343, -3.4982955) (nan, nan)
21 2119 (40.3903716, -3.6348074) (nan, nan)
22 2119 (40.4167754, -3.7037902) (nan, 

In [80]:
number_close_schools = []

for m in range(0, len(resis)):
    number_close_schools.append(len(close_schools[m]))

In [87]:
df = pd.DataFrame(resis, columns = ['resi'])

# close_schools_dist = []
# close_schools = []

# folium search plugin

In [290]:
empty_geojson = {
  "type": "FeatureCollection",
  "features": [ 
  ]}


for i in range(0,len(resis)):
    x = resis.iloc[i]['Residencia']
    long = resis.iloc[i]['longitudes']
    lat = resis.iloc[i]['latitudes']
    
    if lat != 'NaN' and long != 'NaN':
        j = {
        "type": "FeatureCollection",
        "features": [
            {
            "type": "Feature",
            "properties": {"name": x},
            "geometry": {
                "type": "Point",
                "coordinates": [
                long,
                lat
                ]
            }
            }
        ]
        }

        empty_geojson['features'].append(j)

    else: 
        print(i)

    



In [295]:
def write_geojson(fname, coll):
    """Dump GeoJSON FeatureCollection to file"""
    with open(fname, 'w') as f:
        f.write(json.dumps(coll, indent=4, separators=(',', ': ')))

write_geojson('resis_geojson', empty_geojson)


In [307]:
geojson_residencias = gdp.read_file('resis_geojson.geojson', driver= 'GeoJSON')
geo_path = geojson_residencias.geometry.to_json()

In [308]:
points = folium.features.GeoJson(geo_path)
f.add_child(points)

In [309]:
resis_folium_layer = folium.GeoJson(geojson_residencias, name= 'name' ).add_to(f)
# folium.plugins.Search(resis_folium_layer).add_to(f)


In [310]:
resi_search = Search(
    layer=resis_folium_layer,
    geom_type="Point",
    placeholder="Encuentra la residencia",
    collapsed=False,
    search_label="name",
).add_to(f)

In [311]:
folium.LayerControl().add_to(f)
popups, locations = [], []
f

IndexError: list index out of range

In [126]:


def style_function(x):
    return {
        "fillColor": colormap(x["properties"]["density"]),
        "color": "black",
        "weight": 2,
        "fillOpacity": 0.5,
    }


resi_geo = folium.GeoJson(
    pop_ranked_cities,
    name="Residencias de Madrid",
    tooltip=folium.GeoJsonTooltip(
        fields=["nameascii", "pop_max"], aliases=["", "Population Max"], localize=True
    ),
).add_to(f)


resi_search = Search(
    layer=citygeo,
    geom_type="Point",
    placeholder="Encuentra la residencia",
    collapsed=True,
    search_label="nameascii",
).add_to(f)

folium.LayerControl().add_to(f)
colormap.add_to(f)

m

{'type': 'FeatureCollection',
 'features': [{'type': 'FeatureCollection',
   'features': [{'type': 'Feature',
     'properties': {'name': 'Acogida HORTALEZA '},
     'geometry': {'type': 'Point', 'coordinates': [-3.6476904, 40.4728382]}}]},
  {'type': 'FeatureCollection',
   'features': [{'type': 'Feature',
     'properties': {'name': 'Acogida Isabel Clara Eugenia'},
     'geometry': {'type': 'Point', 'coordinates': [-3.6445424, 40.4730365]}}]},
  {'type': 'FeatureCollection',
   'features': [{'type': 'Feature',
     'properties': {'name': 'Acogida casa de campo. Fundación Antonio Moreno'},
     'geometry': {'type': 'Point', 'coordinates': [-3.7491851, 40.415284]}}]},
  {'type': 'FeatureCollection',
   'features': [{'type': 'Feature',
     'properties': {'name': 'Casa de los niños'},
     'geometry': {'type': 'Point', 'coordinates': [-3.7031795, 40.5429909]}}]},
  {'type': 'FeatureCollection',
   'features': [{'type': 'Feature',
     'properties': {'name': 'Residencia Menores Rosa'},
 

# working

In [109]:
colegios = pd.read_csv('./data/data_coles_complete_gm.csv', index_col=0)
resis = pd.read_csv('./data/data_resis_complete_gm.csv', index_col=0)

In [110]:
import pandas as pd
import ipywidgets
from functions import closest_residence, number_close_schools, plot_locations

In [118]:
resi_mas_cercana = closest_residence(colegios, resis)
n_coles_cercanos = number_close_schools(colegios, resis)

/Users/cash/Documents/data_science/alumno/Entregas/Desafio_Tripulaciones/Mapas-Desafio/src/functions.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resi_mas_cercana['distance'] = closest_dist
/Users/cash/Documents/data_science/alumno/Entregas/Desafio_Tripulaciones/Mapas-Desafio/src/functions.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_coles_cercanos['n. coles cercanos'] = numero_coles


In [201]:
colegios = pd.read_csv('./data/data_coles_complete_gm.csv', index_col=0)
resis = pd.read_csv('./data/data_resis_complete_gm.csv', index_col=0)
resi_mas_cercana = pd.read_csv('./data/solos_resis_complete_gm.csv', index_col=0)
n_coles_cercanos = pd.read_csv('./data/n_coles_cercanos.csv', index_col=0)

In [202]:
def plot_map():
    # create map
    plot_locations_map = folium.Map(location=[40.380708,-3.741548], zoom_start=7)

    marker_cluster = MarkerCluster(name="Colegios").add_to(plot_locations_map)

    for i in range(0,len(colegios)):
        try:
            dist = round(dist_array[i],2)
            if dist <= 5:
                icon_path = "./images/book_icon.png"
                icon = folium.features.CustomIcon(icon_image=icon_path, icon_size=(31,29))
                folium.Marker(
                    location=[colegios.iloc[i]['latitudes'],colegios.iloc[i]['longitudes']], 
                    popup= ( colegios.iloc[i]['CENTRO']+': Residencia mas cercana a '+ str(dist)+', '+ resi_array[i]), 
                    label =(colegios.iloc[i]['CENTRO']).lower() , 
                    icon = icon).add_to(marker_cluster)
            else:
                icon_path = "./images/book_icon.png"
                icon2 = folium.features.CustomIcon(icon_image=icon_path ,icon_size=(31,29))
                folium.Marker(
                    location=[colegios.iloc[i]['latitudes'],colegios.iloc[i]['longitudes']], 
                    popup= ( colegios.iloc[i]['CENTRO']+': Residencia mas cercana a '+ str(dist)+', '+ resi_array[i]), 
                    label =(colegios.iloc[i]['CENTRO']).lower(),
                    icon = icon2).add_to(marker_cluster)
        except: 
            x = i

    for i in range(0,len(resis)):
        if resis.iloc[i]['Tipo'] != ('Centro menores con trastornos de salud mental, consucta y/o consumo de sustancias toxicas' or 'Recursos de Adolescentes' or 'Recursos de niños/niñas y adolescentes menores no acompañados'):
            try:
                name = resis.iloc[i]['Residencia'].upper()+' '+ str(numero_coles[i])+', colegios a 5km o menos.'
                folium.Marker(
                    location=[resis.iloc[i]['latitudes'],resis.iloc[i]['longitudes']], 
                    popup=name, 
                    icon=folium.Icon(color='beige', icon_color='white', icon = 'home'), 
                    label =(resis.iloc[i]['Residencia']).lower()
                    ).add_to(plot_locations_map)
            except: 
                try:
                    folium.Marker(
                        location=[resis.iloc[i]['latitudes'],resis.iloc[i]['longitudes']], 
                        popup='No hay colegios a menos de 5km', 
                        icon=folium.Icon(color='beige', 
                        icon_color='#bdd2c1', icon = 'home'), 
                        label =(resis.iloc[i]['Residencia']).lower()
                        ).add_to(plot_locations_map)
                except: 
                    i = i 

    folium.LayerControl().add_to(plot_locations_map)
    
    return plot_locations_map


In [203]:
# using ipywidgets
# plot location with marker

import geocoder
import ipywidgets

# text widget
address_text_box = ipywidgets.Text(value='', placeholder='escribe aqui', description='dirección:')

# widget function
def plot_locations(address):
    # location address
    location = geocoder.osm(address)
    
    # latitude and longitude of location
    latlng = [location.lat, location.lng]
    
    # create map
    plot_locations_map = plot_map()
    
    # marker
    folium.Marker(latlng, popup=str(address), tooltip='click').add_to(plot_locations_map)
    
    # display map
    display(plot_locations_map)
    
# interaction between widget and function  
ipywidgets.interact_manual(plot_locations, address=address_text_box)

interactive(children=(Text(value='', description='dirección:', placeholder='escribe aqui'), Button(description…

<function __main__.plot_locations(address)>